In [ ]:
#####################################################################################################################
#                                                                                                                   #
# Libraries                                                                                                         #
#                                                                                                                   #
#####################################################################################################################
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.rcParams['figure.dpi'] = 200 # Plot resolution (dpi)
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import random
import matplotlib.dates as mdates
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.vector_ar.var_model import VAR # Import VAR class
import shap




In [ ]:

# str_Dir_Plan_Data = '/Users/bodamjerry/Desktop/Oldenburg Studies/Winter2425/Applied Economics/data/PC/'

str_Dir_Plan_FRED = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/FRED/'
str_Dir_Plan_Data = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PCC/'
str_Dir_Plan_PC = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXNew/'

str_Nome_Plan_FRED_MD = 'current'
str_FRED_MD_Desc = 'Data_Description_MD.csv'

X = pd.read_csv(filepath_or_buffer = str_Dir_Plan_PC + '0 ' + 'X.csv')
y = pd.read_csv(filepath_or_buffer = str_Dir_Plan_PC + '0 ' + 'y.csv')

df_FRED_MD = pd.read_csv(filepath_or_buffer = str_Dir_Plan_FRED + str_Nome_Plan_FRED_MD + '.csv', skiprows=[1])

# print(X.head())
# print(y.head())
df_date = X['Date']
# data = X[['UNRATE', 'RETAILx', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
data_t = pd.concat([X, y['CPIAUCSL']], axis=1)
#
data_t.set_index('Date', inplace=True)
# data_t.drop('Date', axis=1, inplace=True)
# data = data_t[['CPIAUCSL', 'UNRATE', 'HOUST', 'HOUSTNE', 'PERMITNE', 'AAA', 'BAA', 'PERMITMW', 'RETAILx', 'TB3MS', 'TB6MS', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
data = data_t.copy()

print(data.shape)
print(data.columns)

print(data.head())
print(data.tail())


# var_model.summary()

In [ ]:


def display_variable_descriptions(data_columns, description_file_path='Data_Description_MD.csv'):
    """
    Displays the description and group of each variable in data_columns
    by looking up information in the 'Data_Description_MD.csv' file.

    Args:
        data_columns (Index): pandas Index object containing column names.
        description_file_path (str, optional): Path to the Data Description CSV file.
                                                Defaults to 'Data_Description_MD.csv'.
    """

    try:
        description_df = pd.read_csv(description_file_path, delimiter=';')
    except FileNotFoundError:
        print(f"Error: Description file not found at '{description_file_path}'.")
        return

    print("Variable Descriptions and Groups:")
    print("-" * 40)

    for col_name in data_columns:
        # Find the row in description_df where 'FRED' column matches the current column name
        variable_info = description_df[description_df['FRED'] == col_name]

        if not variable_info.empty:
            description = variable_info['Description'].iloc[0]  # Get the description from the first row
            group = variable_info['Group'].iloc[0]          # Get the group from the first row
            print(f"Variable: {col_name}")
            print(f"  Description: {description}")
            print(f"  Group: {group}")
        else:
            print(f"Variable: {col_name}")
            print(f"  Description: Description not found in '{description_file_path}'")
            print(f"  Group: Group not found in '{description_file_path}'")
        print("-" * 40)

# Assuming you have 'data' DataFrame and 'data.columns' is available
# and the 'Data_Description_MD.csv' file is in the same directory or you provide the correct path.

# Example Usage (assuming 'data_l' is your loaded DataFrame):
# After you run the code that loads your data and prints data.columns,
# and you have the 'data' DataFrame available (e.g., data = data_l[all_variables].copy())
# Uncomment and run the following lines:

path = str_Dir_Plan_FRED + str_FRED_MD_Desc
display_variable_descriptions(data.columns, path)

In [ ]:

def get_variable_descriptions(col_name, description_file_path='Data_Description_MD.csv'):

    try:
        description_df = pd.read_csv(description_file_path, delimiter=';')
    except FileNotFoundError:
        print(f"Error: Description file not found at '{description_file_path}'.")
        return

    # print("Variable Descriptions and Groups:")
    # print("-" * 40)

    # Find the row in description_df where 'FRED' column matches the current column name
    variable_info = description_df[description_df['FRED'] == col_name]

    if not variable_info.empty:
        description = variable_info['Description'].iloc[0]  # Get the description from the first row
        group = variable_info['Group'].iloc[0]          # Get the group from the first row
        return f"{col_name}: {group}"

    else:
        print(f"Variable: {col_name}")
        # print(f"  Description: Description not found in '{description_file_path}'")
        # print(f"  Group: Group not found in '{description_file_path}'")

# # print(data.index)
# for col_name in data.columns:
#     plot_series(data[col_name], get_variable_descriptions(col_name, path))


sns.set_theme(style="whitegrid")

def plot_grid_series(data, column_names=None, grid_rows=4, grid_cols=3, figsize=(18, 14)):
    """
    Plots time series for multiple columns in a DataFrame in a grid layout.

    Args:
        data (pd.DataFrame): DataFrame containing time series data.
        column_names (list, optional): List of column names to plot.
                                        If None, all columns in data are plotted. Defaults to None.
        grid_rows (int, optional): Number of rows in the subplot grid. Defaults to 4.
        grid_cols (int, optional): Number of columns in the subplot grid. Defaults to 3.
        figsize (tuple, optional): Figure size for the entire grid plot. Defaults to (18, 14).
    """

    if column_names is None:
        column_names = data.columns

    num_plots = len(column_names)
    fig, axes = plt.subplots(nrows=grid_rows, ncols=grid_cols, figsize=figsize)
    axes = axes.flatten() # Flatten the 2D array of axes to easily iterate

    for i, col_name in enumerate(column_names):
        series = data[col_name]
        ax = axes[i] # Get the current subplot axes

        # --- Ensure series.index is a DateTimeIndex ---
        if not isinstance(series.index, pd.DatetimeIndex):
            try:
                series.index = pd.to_datetime(series.index)
            except ValueError:
                print(f"Warning: Series index for {col_name} not in DateTime format. Plotting without date formatting.")
                ax.plot(series.values, label=col_name, color='#2b8cbe', linestyle='-', linewidth=1.5)
                ax.set_title(col_name, fontsize=14) # Smaller subplot title font
                ax.set_xlabel('Time', fontsize=10)   # Smaller subplot xlabel font
                ax.set_ylabel('Value', fontsize=10)  # Smaller subplot ylabel font
                ax.tick_params(axis='x', rotation=45, labelsize=8) # Smaller tick label font
                ax.grid(True, linestyle='--', alpha=0.5, color='gray')
                ax.legend(fontsize='small', frameon=True, edgecolor='black')
                continue # Skip date formatting for this subplot

        ax.plot(series.index, series.values, label=col_name, color='#2b8cbe', linestyle='-', linewidth=1.5)

        # --- Date Locator and Formatter for each subplot ---
        locator = mdates.AutoDateLocator()
        ax.xaxis.set_major_locator(locator)
        formatter = mdates.DateFormatter('%Y-%m')
        ax.xaxis.set_major_formatter(formatter)

        ax.set_title(get_variable_descriptions(col_name, path), fontsize=14) # Smaller subplot title font
        ax.set_xlabel('Date', fontsize=10)   # Smaller subplot xlabel font
        ax.set_ylabel('Value', fontsize=10)  # Smaller subplot ylabel font

        ax.tick_params(axis='x', rotation=45, labelsize=8) # Smaller tick label font
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')
        ax.legend(fontsize='small', frameon=True, edgecolor='black')

        # --- Customize Spines for each subplot ---
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(0.5)
        ax.spines['left'].set_linewidth(0.5)


    # Remove any unused subplots if num_plots is less than grid_rows * grid_cols
    for j in range(num_plots, grid_rows * grid_cols):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.97]) # Adjust layout for better spacing, make room for suptitle
    plt.suptitle('Time Series Grid', fontsize=18) # Overall title for the grid
    plt.show()


# --- Example Usage ---
# Assuming 'data' is your DataFrame with 12 columns
plot_grid_series(data, grid_rows=3, grid_cols=4) # Plots all columns in a 4x3 grid (default)

# If you want to plot only specific columns (e.g., first 6 columns in a 2x3 grid):
# columns_to_plot = data.columns[:6]
# plot_grid_series(data[columns_to_plot], column_names=columns_to_plot, grid_rows=2, grid_cols=3)


In [ ]:
data_t = data.copy()
def create_lag_features(df, columns, lags):
    """
    Creates lag features for specified columns in a pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to add lag features to.
        columns (list): A list of column names for which to create lag features.
        lags (int): The number of lags to create.

    Returns:
        pd.DataFrame: The DataFrame with added lag features.
    """
    for col in columns:
        for i in range(1, lags + 1):
            df[f'{col}_L{i}'] = df[col].shift(i)
    return df

save_dir = "/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXShuffle"

# Uncomment to create random state

# random.seed(123)
# for i in range(100):
#     random_state = random.randint(1, 1000)
# 
#     # Shuffle data_t and save as CSV
#     shuffled_data = data_t.sample(frac=1, random_state=random_state).reset_index(drop=True)
#     shuffled_data.to_csv(os.path.join(save_dir, f"shuffled_data_{i+1}.csv"), index=False)
# 
#     # Create data DataFrame with selected columns
# 
# 

# data = data_t[['CPIAUCSL', 'UNRATE', 'RETAILx', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
data.drop('HOUSTNE', axis=1, inplace=True)



In [ ]:
# data.drop()

# test_size = int(len(data) * 0.2)  # Convert percentage to absolute count
test_size = 12
train_df = data.iloc[:-test_size]
test_df = data.iloc[-test_size:]

print(test_df.shape)
print(train_df.shape)

print(data_t.shape)
print(data.shape)

In [ ]:
def check_stationarity(series, column_name):
    """Performs ADF test and prints results."""
    result = adfuller(series)
    print(f'ADF Test Results for: {column_name}')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    if result[1] <= 0.05:
        print(f"{column_name} is likely stationary.")
    else:
        print(f"{column_name} is likely non-stationary.")
    print("-" * 50)

# Check stationarity for all columns
for col in data.columns:
    check_stationarity(data[col], col)

# If any series are non-stationary, apply differencing
data_diff = data.diff().dropna() # First-order differencing

# Re-check stationarity after differencing
print("\nStationarity after Differencing:")
for col in data_diff.columns:
    check_stationarity(data_diff[col], col)

In [ ]:
model = VAR(train_df)

# AIC based lag order (your original code)
lag_order_aic = model.select_order(maxlags=15).aic
print(f"AIC-based Lag Order: {lag_order_aic}")

# BIC based lag order
lag_order_bic = model.select_order(maxlags=15).bic
print(f"BIC-based Lag Order: {lag_order_bic}")

# HQIC based lag order
lag_order_hqic = model.select_order(maxlags=15).hqic
print(f"HQIC-based Lag Order: {lag_order_hqic}")

# Choose the lag order based on the criteria you prefer or by comparing performance with different lag orders.
lag_order = lag_order_aic # or lag_order_bic, or lag_order_hqic
var_model = model.fit(lag_order)
print(lag_order)

var_model.summary()


In [ ]:
# Forecast the next 12 months

forecast = var_model.forecast(train_df.values[-lag_order:], steps=test_size)
# predict = var_model.get_prediction(start=train_df.shape[0], end=train_df.shape[0] + 11)
forecast_df = pd.DataFrame(forecast, columns=train_df.columns)

# Extract forecasted CPIAUCSL 
forecasted_cpiauscl = forecast_df['CPIAUCSL']

actual_cpiauscl = test_df['CPIAUCSL']

mae = mean_absolute_error(actual_cpiauscl, forecasted_cpiauscl)
rmse = np.sqrt(mean_squared_error(actual_cpiauscl, forecasted_cpiauscl))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


sns.set_theme(style="whitegrid") # Apply seaborn visual theme

def plot_forecast(forecasted_series, actual_series, title_name='Forecasted vs Actual CPIAUCSL'):

    fig = plt.figure(figsize=(12, 6)) # Store figure object

    actual_series.index = pd.to_datetime(actual_series.index) # Try converting to DateTimeIndex

    plt.plot(actual_series.index, forecasted_series, label='Forecasted', color='#e41a1c', linestyle='dashed', linewidth=2) # Red for forecast
    plt.plot(actual_series.index, actual_series, label='Actual', color='#377eb8', linestyle='solid', linewidth=2) # Blue for actual

    plt.xlabel('Date', fontsize=10)
    plt.ylabel('Value', fontsize=10)
    plt.title(title_name, fontsize=12)

    # Date formatting for x-axis (using logic from your improved plot_series)
    ax = plt.gca()
    locator = mdates.AutoDateLocator()
    ax.xaxis.set_major_locator(locator)
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=90, ha='right', fontsize=10) # Rotate x-axis labels, adjust font size
    plt.gcf().autofmt_xdate() # Auto-format dates

    plt.legend(fontsize=10, frameon=True, shadow=False, edgecolor='black')
    plt.grid(True, linestyle='--', alpha=0.7, color='gray')

    # Customize Spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)

    plt.tight_layout() # Adjust layout

    return fig # Return the figure object (do NOT use plt.show() here if you want to control display later)


def plot_observed_vs_fitted(var_model_fit, train_df, target_column='CPIAUCSL', title='VAR Model - Observed vs Fitted (Training)'):

    fitted_values = var_model_fit.fittedvalues # Get fitted values from the VAR model
    observed_train = train_df[target_column]   # Observed values from the training data

    # Extract fitted values for the target column
    fitted_cpiauscl = fitted_values[target_column]

    # Ensure index is DateTimeIndex if possible (align with actual_series index if needed)
    try:
        fitted_cpiauscl.index = pd.to_datetime(fitted_cpiauscl.index)
        observed_train.index = pd.to_datetime(observed_train.index) # Align observed index too for plotting
        if not fitted_cpiauscl.index.equals(observed_train.index): # If indexes still not the same, try to align based on actual index
            fitted_cpiauscl.index = observed_train.index # Force index alignment for plotting if possible
    except ValueError:
        print("Warning: Could not convert index to DateTime for Observed vs Fitted plot.")
        pass # Plot without DateTimeIndex if conversion fails

    fig = plt.figure(figsize=(12, 6))

    plt.plot(observed_train.index, observed_train, label="Observed (Train)", linestyle="solid", linewidth=2) # Blue for observed
    plt.plot(fitted_cpiauscl.index, fitted_cpiauscl, label="Fitted (Train)", linestyle="solid", linewidth=2) # Green for fitted

    plt.title(title, fontsize=12)
    plt.xlabel("Date", fontsize=10)
    plt.ylabel("Value", fontsize=10)
    plt.legend(fontsize=10, frameon=True, shadow=False, edgecolor='black')
    plt.grid(True, linestyle='--', alpha=0.7, color='gray')

    # Date formatting for x-axis
    ax = plt.gca()
    locator = mdates.AutoDateLocator() # Use observed_train index for locator
    ax.xaxis.set_major_locator(locator)
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.gcf().autofmt_xdate()

    # Customize Spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)

    plt.tight_layout()
    plt.show()
    return fig

# Generate the plot by calling the function:
var_model_forecast_plot = plot_forecast(forecasted_cpiauscl, actual_cpiauscl, title_name='VAR Model Forecast - CPIAUCSL')
var_observed_vs_fitted_plot = plot_observed_vs_fitted(var_model, train_df, target_column='CPIAUCSL', title='VAR Model - Observed vs Fitted (Training)')

# To display the plot immediately (if you are not planning to put it in a grid right now):
plt.show() # Call plt.show() after plot_forecast if you want to display it now

# --- Later, if you want to put this plot in a grid ---
# You would collect these figure objects from different models (ARIMA, SARIMA, CNN-LSTM etc.)
# and then use them in your plot_grid_series function (or a modified version) to arrange them in a grid.



In [ ]:


# 1. ARIMA Model for CPIAUCSL

# Extract CPIAUCSL series for ARIMA
train_cpiauscl_arima = train_df['CPIAUCSL']
test_cpiauscl_arima = test_df['CPIAUCSL']

In [ ]:
# For ARIMA order selection (non-seasonal):
auto_arima_model_arima = auto_arima(train_cpiauscl_arima, seasonal=False, stepwise=True,
                                    suppress_warnings=True, error_action="ignore", max_order=10, trace=True)
print(auto_arima_model_arima.order) # Get the recommended (p, d, q) order



In [ ]:
# For SARIMA order selection (seasonal):
auto_arima_model_sarima = auto_arima(train_cpiauscl_arima, seasonal=True, m=12, stepwise=True, # m=12 for monthly seasonality
                                     suppress_warnings=True, error_action="ignore", max_order=10, trace=True,
                                     max_p=5, max_q=5, max_P=5, max_Q=5) # Limit search space
print(auto_arima_model_sarima.order) # Get recommended (p, d, q)
print(auto_arima_model_sarima.seasonal_order) # Get recommended (P, D, Q, s)

In [ ]:

def plot_arima_observed_vs_fitted(model_fit, train_data, target_column='CPIAUCSL', title='VAR Model - Observed vs Fitted (Training)'):


    fitted_series = model_fit.fittedvalues # For ARIMA, fittedvalues is already a Series for the fitted series

    observed_train = train_data[target_column] if isinstance(train_data, pd.DataFrame) else train_data # Handle DataFrame or Series input for train_data

    # Extract fitted values for the target column
    # fitted_cpiauscl = fitted_values[target_column]
    fig = plt.figure(figsize=(12, 6))

    # Ensure index is DateTimeIndex if possible (align with actual_series index if needed)
    plt.plot(observed_train.index, observed_train, label="Observed (Train)", linestyle="solid", linewidth=2) # Blue for observed
    plt.plot(fitted_series.index, fitted_series, label="Fitted (Train)", linestyle="solid", linewidth=2) # Green for fitted

    plt.title(title, fontsize=10)
    plt.xlabel("Date", fontsize=10)
    plt.ylabel("Value", fontsize=10)
    plt.legend(fontsize=10, frameon=True, shadow=False, edgecolor='black')
    plt.grid(True, linestyle='--', alpha=0.7, color='gray')

    # Date formatting for x-axis
    ax = plt.gca()
    locator = mdates.AutoDateLocator() # Use observed_train index for locator
    ax.xaxis.set_major_locator(locator)
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.gcf().autofmt_xdate()

    # Customize Spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)

    plt.tight_layout()
    plt.show()
    return fig


In [ ]:


# Assuming 'data' DataFrame is already loaded and preprocessed
# and train_df, test_df are already created as in your VAR code

# 1. ARIMA Model for CPIAUCSL

# ARIMA Model Order (p, d, q) - You'll need to determine this based on ACF, PACF, or auto_arima
# For demonstration, let's assume a simple order (you should optimize this)
# arima_order = (5, 1, 0) # Example order - Autoregressive order (p=5), Integrated (d=1), Moving Average (q=0)

# Fit ARIMA model
arima_model = ARIMA(train_cpiauscl_arima, order=auto_arima_model_arima.order)
arima_model_fit = arima_model.fit()

# Forecast for test period (24 steps)
arima_forecast = arima_model_fit.forecast(steps=test_size)

# Evaluate ARIMA model
mae_arima = mean_absolute_error(test_cpiauscl_arima, arima_forecast)
rmse_arima = np.sqrt(mean_squared_error(test_cpiauscl_arima, arima_forecast))

print(f'ARIMA Model - Mean Absolute Error (MAE): {mae_arima:.4f}')
print(f'ARIMA Model - Root Mean Squared Error (RMSE): {rmse_arima:.4f}')

# Plot ARIMA Forecast
# plt.figure(figsize=(10, 6))
# plt.plot(test_cpiauscl_arima.index, arima_forecast, label='ARIMA Forecasted CPIAUCSL', color='blue')
# plt.plot(test_cpiauscl_arima.index, test_cpiauscl_arima, label='Actual CPIAUCSL', color='red', linestyle='--')
# plt.xlabel('Time')
# plt.ylabel('CPIAUCSL')
# plt.title('ARIMA Forecasted vs Actual CPIAUCSL')
# plt.legend()
# plt.show()

arima_forecast = plot_forecast(arima_forecast, actual_cpiauscl, title='ARIMA: Forecasted vs Actual')
arima_fitted = plot_arima_observed_vs_fitted(arima_model_fit, train_df, title='ARIMA: Observed vs Fitted')
plt.show()



In [ ]:

# 2. SARIMA Model for CPIAUCSL

# Extract CPIAUCSL series (already done above: train_cpiauscl_arima, test_cpiauscl_arima)

# SARIMA Model Order (p, d, q) and Seasonal Order (P, D, Q, s)
# You'll need to determine these based on ACF, PACF, seasonal decomposition, or auto_arima
# For demonstration, let's assume simple orders (you should optimize this)
# sarima_order = (5, 1, 0)          # (p, d, q) - Non-seasonal order - same as ARIMA for simplicity in example
# sarima_seasonal_order = (1, 1, 0, 12) # (P, D, Q, s) - Seasonal order, s=12 for yearly seasonality (monthly data)

# Fit SARIMA model
sarima_model = SARIMAX(train_cpiauscl_arima,
                       order=auto_arima_model_sarima.order,
                       seasonal_order=auto_arima_model_sarima.seasonal_order)
sarima_model_fit = sarima_model.fit(disp=False) # disp=False to suppress convergence output

# Forecast for test period (24 steps)
sarima_forecast = sarima_model_fit.forecast(steps=test_size)

# Evaluate SARIMA model
mae_sarima = mean_absolute_error(test_cpiauscl_arima, sarima_forecast)
rmse_sarima = np.sqrt(mean_squared_error(test_cpiauscl_arima, sarima_forecast))


sarima_forecast = plot_forecast(sarima_forecast, actual_cpiauscl)
sarima_fitted = plot_arima_observed_vs_fitted(sarima_model_fit, train_df)
plt.show()

print(f'SARIMA Model - Mean Absolute Error (MAE): {mae_sarima:.4f}')
print(f'SARIMA Model - Root Mean Squared Error (RMSE): {rmse_sarima:.4f}')

# model = VAR(train_df)
# var_model = model.fit(lag_order)
# forecast = var_model.forecast(train_df.values[-lag_order:], steps=12)
# forecast_df = pd.DataFrame(forecast, columns=train_df.columns)
# 
# forecasted_cpiauscl = forecast_df['CPIAUCSL']
# actual_cpiauscl = test_df['CPIAUCSL']


In [ ]:
# Check for serial correlation in residuals
from statsmodels.stats.stattools import durbin_watson
dw_test_results = durbin_watson(var_model.resid)
print("\nDurbin-Watson Test for Residual Autocorrelation:")
for col, dw in zip(train_df.columns, dw_test_results):
    print(f"{col}: DW Statistic = {dw:.4f} (Ideal: ~2)")

In [ ]:

# Directory to save CSV files
save_dir = "/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXShuffle"

def train_and_evaluate_var_model(data, lag_order):
    """
    Trains and evaluates a VAR model on a given dataset.

    Args:
        data (pd.DataFrame): DataFrame with lagged time series data.
        lag_order (int): The lag order for the VAR model.

    Returns:
        tuple: MAE, RMSE, and forecasted CPIAUCSL values.
    """

    train_df = data.iloc[:-12]
    test_df = data.iloc[-12:]

    model = VAR(train_df)
    var_model = model.fit(lag_order)
    forecast = var_model.forecast(train_df.values[-lag_order:], steps=12)
    forecast_df = pd.DataFrame(forecast, columns=train_df.columns)

    forecasted_cpiauscl = forecast_df['CPIAUCSL']
    actual_cpiauscl = test_df['CPIAUCSL']

    mae = mean_absolute_error(actual_cpiauscl, forecasted_cpiauscl)
    rmse = np.sqrt(mean_squared_error(actual_cpiauscl, forecasted_cpiauscl))

    return mae, rmse, forecasted_cpiauscl

# Perform robust testing with 50 random states
var_maes = []
var_rmses = []
var_all_forecasts = []

for i in range(100):
    # Read shuffled data from CSV file
    file_path = os.path.join(save_dir, f"shuffled_data_{i+1}.csv")
    shuffled_data = pd.read_csv(file_path)

    # Create data DataFrame with selected columns
    # data = shuffled_data[['CPIAUCSL_1', 'UNRATE', 'RETAILx', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
    data = shuffled_data.copy()
    mae, rmse, forecasted_cpiauscl = train_and_evaluate_var_model(data, 9)
    var_maes.append(mae)
    var_rmses.append(rmse)
    var_all_forecasts.append(forecasted_cpiauscl)

# ... (rest of your code for calculating averages and plotting)

In [ ]:

# Calculate and print average MAE and RMSE for the VAR model
avg_var_mae = np.mean(var_maes)
avg_var_rmse = np.mean(var_rmses)

print(f'VAR Average MAE: {avg_var_mae}')
print(f'VAR Average RMSE: {avg_var_rmse}')

# Calculate standard deviation for MAE and RMSE
std_var_mae = np.std(var_maes)
std_var_rmse = np.std(var_rmses)

# Plotting the MAEs and RMSEs with shaded confidence intervals
plt.figure(figsize=(14, 7))

# Plot VAR MAEs with shaded confidence interval
plt.plot(range(1, 101), var_maes, label='VAR MAE', marker='o', linestyle='-', color='blue', alpha=0.8)
plt.fill_between(range(1, 101),
                 np.array(var_maes) - std_var_mae,
                 np.array(var_maes) + std_var_mae,
                 color='blue', alpha=0.2, label='MAE Confidence Interval')

# Add horizontal lines for average MAE and RMSE
plt.axhline(y=avg_var_mae, color='blue', linestyle='--', alpha=0.5, label=f'Avg VAR MAE: {avg_var_mae:.2f}')
# plt.axhline(y=avg_var_rmse, color='green', linestyle='--', alpha=0.5, label=f'Avg VAR RMSE: {avg_var_rmse:.2f}')

# Add annotations for average values
plt.text(101, avg_var_mae, f'Avg MAE: {avg_var_mae:.2f}', color='blue', va='center', ha='left')
# plt.text(101, avg_var_rmse, f'Avg RMSE: {avg_var_rmse:.2f}', color='green', va='center', ha='left')

# Customize the plot
plt.title('VAR: MAE and RMSE over 100 Random States with Confidence Intervals', fontsize=16)
plt.xlabel('Random State', fontsize=14)
plt.ylabel('Error', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
data_l = data.copy()
# data_l = data_l.drop('CPIAUCSL', axis=1)

# print(data_t.head());
print(data_l.head());
# print(data_l.tail());

In [ ]:
columns_to_lag = ['CPIAUCSL_1', 'UNRATE', 'RETAILx', 'FEDFUNDS', 'S&P 500', 'CP3Mx']
# data_ls = create_lag_features(data_l, columns_to_lag, lags=5)
# data_ls.dropna(inplace=True)
print(data_l.head())

# print(data_ls.head())

In [ ]:

def prepare_data_for_cnn_lstm(data_l, target_col='CPIAUCSL', forecast_horizon=1, lags=3, test_size=0.2):

    # Copy original data to avoid modifying in place
    data_x = data_l.copy()
    # print(data_x.head())

    # Initialize scalers
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    # Separate target and features
    y = data_x[[target_col]]
    X = data_x.drop(target_col, axis=1)

    # Scale the features and target
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    # Convert back to DataFrame (preserving column names and index)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    y_scaled_df = pd.DataFrame(y_scaled, columns=y.columns, index=y.index)

    # Concatenate scaled features and target
    data = pd.concat([X_scaled_df, y_scaled_df], axis=1)

    # Shift the target column to align with forecasting
    data[target_col+ '1'] = data_x[target_col]
    data[target_col] = data_x[target_col].shift(-forecast_horizon)

    columns_to_lag = [col for col in data.columns if col != target_col] # All columns except target
    # Create lagged features
    data_ls = create_lag_features(data, columns_to_lag, lags=lags)
    data_ls.dropna(inplace=True)
    return data_ls, scaler_X, scaler_y

# Example usage
# X_train, y_train, X_test, y_test, scaler_X, scaler_y, feature_c = prepare_data_for_cnn_lstm(data_l, 'CPIAUCSL')
data_ls, scaler_X, scaler_y = prepare_data_for_cnn_lstm(data_l, 'CPIAUCSL')
# print(data_ls.head())
# print(data_ls.columns)
print(data_ls.shape)

In [ ]:

def prepare_data(data, target_column='CPIAUCSL', test_size=12, test_size_percent=False):
    """
    Splits the data into training and testing sets and reshapes it for ConvLSTM2D.

    Parameters:
    - data (DataFrame): The input dataset.
    - target_column (str): The column name of the target variable.
    - test_size (int or float): Number of test samples (if int) or percentage of data for testing (if float).
    - test_size_percent (bool): If True, interprets test_size as a percentage (0-1).

    Returns:
    - X_train, y_train, X_test, y_test: Training and testing datasets, reshaped for ConvLSTM2D.
    """
    if test_size_percent:
        test_size = int(len(data) * test_size)  # Convert percentage to absolute count

    print(data.head())

    train_df = data.iloc[:-test_size]
    test_df = data.iloc[-test_size:]
    train_index = train_df.index
    test_index = test_df.index

    X_train = train_df.drop(target_column, axis=1).values
    y_train = train_df[target_column].values
    X_test = test_df.drop(target_column, axis=1).values
    y_test = test_df[target_column].values

    X_train = X_train.reshape((X_train.shape[0], 1, 1, 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, 1, 1, X_test.shape[1]))

    return X_train, y_train, X_test, y_test, train_index, test_index

def build_model(input_shape):
    """Builds and compiles a ConvLSTM2D model."""
    model = Sequential([
        Bidirectional(ConvLSTM2D(filters=128, kernel_size=(1, 1), activation='selu', return_sequences=True, input_shape=input_shape)),
        Bidirectional(ConvLSTM2D(filters=64, kernel_size=(1, 1), activation='selu', return_sequences=False)),
        Flatten(),
        Dense(1)
    ])

    opt = Adam(learning_rate=0.0005)
    model.compile(optimizer=opt, loss='mse')
    return model

def train_model(model, X_train, y_train):
    """Trains the model with early stopping."""
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=300, batch_size=288, validation_split=0.2, callbacks=[early_stopping])
    return history

def evaluate_model(model, X_test, y_test):
    """Evaluates the model and returns the test loss and predictions."""
    loss = model.evaluate(X_test, y_test, verbose=0)
    predictions = model.predict(X_test, verbose=0)
    return loss, predictions

def inverse_transform(scaler, data):
    """Inverse transforms the scaled data."""
    return scaler.inverse_transform(data.reshape(-1, 1))

def plot_predictions(y_actual, y_pred, train_index, title='CNN-LSTM: Actual vs Forecasted Values'):
    """Plots actual vs forecasted values."""
    train_index = pd.to_datetime(train_index)
    fig = plt.figure(figsize=(12, 6))
    plt.plot(train_index, y_actual, label='Actual', color='blue', linestyle='solid')
    plt.plot(train_index, y_pred, label='Forecasted', color='red', linestyle='dashed')

    ax = plt.gca()
    locator = mdates.AutoDateLocator()
    ax.xaxis.set_major_locator(locator)
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.gcf().autofmt_xdate()

    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title(title, fontsize=10)
    plt.legend()
    plt.grid(True)
    plt.show()
    return fig

def plot_loss(history):
    # Plots loss function
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LSTM - Loss (MSE)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.show()
    plt.close()

def plot_observed_vs_fitted(y_train, predictions_train, train_index):
    fig = plt.figure(figsize=(12, 6))
    train_index = pd.to_datetime(train_index)
    plt.plot(train_index, y_train, label="Observed (Train)", linestyle="solid")
    plt.plot(train_index, predictions_train, label="Fitted (Train)", linestyle="solid")
    plt.title("CNN-LSTM - Observed vs. Fitted Values")

    ax = plt.gca()
    locator = mdates.AutoDateLocator()
    ax.xaxis.set_major_locator(locator)
    formatter = mdates.DateFormatter('%Y-%m')
    ax.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.gcf().autofmt_xdate()

    plt.tight_layout()
    plt.xlabel("Time Steps")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()
    return fig

def main(data, scaler_y):
    """Main function to execute the pipeline."""
    X_train, y_train, X_test, y_test, train_index, test_index = prepare_data(data, test_size=12, test_size_percent=False)
    model = build_model((1, 1, 1, X_train.shape[-1]))
    history = train_model(model, X_train, y_train)

    loss, predictions = evaluate_model(model, X_test, y_test)
    print(f'Test Loss: {loss}')

    y_test_actual = inverse_transform(scaler_y, y_test)
    predictions_actual = inverse_transform(scaler_y, predictions)

    mae = mean_absolute_error(y_test_actual, predictions_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions_actual))
    plot_cnn_forecast = plot_predictions(y_test_actual, predictions_actual, test_index)
    # plot_loss(history)

    predictions_train = model.predict(X_train, verbose=0) # Predict on X_train
    predictions_train_actual = inverse_transform(scaler_y, predictions_train) # Inverse transform fitted values
    y_train_actual = inverse_transform(scaler_y, y_train) # Inverse transform y_train

    # --- Plot Observed vs Fitted on Training Data ---
    plot_cnn_obs_vs_fitted = plot_observed_vs_fitted(y_train_actual, predictions_train_actual, train_index) # Pass y_train_actual and predictions_train_actual

    # plot_observed_vs_fitted(y_train, predictions)

    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')
    return plot_cnn_forecast, plot_cnn_obs_vs_fitted, model, X_train, X_test

# Call main(data_lst, scaler_y) with your actual data and scaler
# plot_cnn_forecast, plot_cnn_obs_vs_fitted, model, X_train, X_test = main(data_ls.copy(), scaler_y)
prepare_data(data_ls[['CPIAUCSL1', 'CPIAUCSL', 'CPIAUCSL1_L1', 'CPIAUCSL1_L2', 'CPIAUCSL1_L3']])


In [ ]:

# Wrapper function for SHAP
def model_predict(X):
    """Predict function for SHAP that ensures correct input shape."""
    X = X.reshape((X.shape[0], 1, 1, 1, X.shape[1]))  # Reshape to original model input shape
    return model.predict(X, verbose=0).flatten()  # Flatten output for SHAP

# Select a subset of X_test for SHAP analysis (SHAP can be slow)
feature_names = data_ls.drop(columns=['CPIAUCSL1']).columns.tolist()
num_samples = min(50, X_test.shape[0])  # Ensure we don't exceed the available samples

X_sample = X_test[:num_samples].reshape((num_samples, X_test.shape[-1]))  # Convert to 2D
X_sample_df = pd.DataFrame(X_sample, columns=feature_names)  # Convert NumPy array back to DataFrame
# All Sample
# X_sample_df = pd.DataFrame(X_test.reshape(X_test.shape[0], X_test.shape[-1]), columns=feature_names) 


# Create Kernel SHAP Explainer
explainer = shap.KernelExplainer(model_predict, X_sample_df)

# Compute SHAP values
shap_values = explainer.shap_values(X_sample_df)

# Create SHAP summary plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust figure size if needed
shap.summary_plot(shap_values, X_sample_df, show=False)  # Prevent immediate display

# Adjust font sizes
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(8)  # Change this value to your desired font size

plt.show()


# Plot summary plot
# shap.summary_plot(shap_values, X_sample_df)


In [ ]:

def plot_grid_series(figures, titles, grid_rows=2, grid_cols=4, figsize=(20, 10)):
    """
    Plots a grid of time series plots.

    Args:
        figures (list of plt.Figure): List of matplotlib Figure objects to be plotted in the grid.
        titles (list of str): List of titles for each subplot, corresponding to the figures.
        grid_rows (int): Number of rows in the grid.
        grid_cols (int): Number of columns in the grid.
        figsize (tuple): Figure size for the entire grid plot.
    """
    fig, axes = plt.subplots(nrows=grid_rows, ncols=grid_cols, figsize=figsize)
    axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

    for i, fig_obj in enumerate(figures):
        ax = fig_obj.axes[0] # Assuming each figure has one axes (common for these plots)
        target_ax = axes[i]

        # Copy lines from the original axes to the subplot
        for line in ax.lines:
            target_ax.plot(line.get_xdata(), line.get_ydata(),
                           color=line.get_color(), linestyle=line.get_linestyle(),
                           label=line.get_label(), linewidth=line.get_linewidth())

        # Copy title, labels, legend from the original axes
        target_ax.set_title(titles[i])
        target_ax.set_xlabel(ax.get_xlabel())
        target_ax.set_ylabel(ax.get_ylabel())

        # Handle legend - only if there are lines with labels
        if ax.get_legend() is not None:
            target_ax.legend()

        target_ax.grid(True, linestyle='--', alpha=0.7, color='gray') # Ensure grid is on in subplots

        # Date formatting - try to apply if x-axis is date-like
        try:
            target_ax.xaxis.set_major_locator(ax.xaxis.get_major_locator())
            target_ax.xaxis.set_major_formatter(ax.xaxis.get_major_formatter())
            for label in target_ax.get_xticklabels():
                label.set_ha("right")
                label.set_rotation(45)
        except AttributeError:
            pass # If no date formatting, ignore error

        # Customize Spines - copy spine settings if needed (optional, can be customized for grid)
        target_ax.spines['top'].set_visible(False)
        target_ax.spines['right'].set_visible(False)
        target_ax.spines['bottom'].set_linewidth(0.5)
        target_ax.spines['left'].set_linewidth(0.5)


    plt.tight_layout()
    plt.show()


# --- Example Usage: ---
plot_figures = [
    var_model_forecast_plot,
    var_observed_vs_fitted_plot,
    arima_forecast,
    arima_fitted,

    sarima_forecast,
    sarima_fitted,
    plot_cnn_forecast,
    plot_cnn_obs_vs_fitted
]

plot_titles = [
    'VAR - Forecast',
    'VAR - Observed vs Fitted',
    'ARIMA - Forecast',
    'ARIMA - Fitted',
    'SARIMA - Forecast',
    'SARIMA - Fitted',
    'CNN-LSTM - Forecast',
    'CNN-LSTM - Observed vs Fitted'
]

plot_grid_series(plot_figures, plot_titles)



In [ ]:
 def prep_data_for_cnn(data_l, target_col='CPIAUCSL', forecast_horizon=1, lags=3, test_size=12, test_size_percent=False):

    # Copy original data to avoid modifying in place
    data_x = data_l.copy()
    # print(data_x.head())

    # Initialize scalers
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    # Separate target and features
    y = data_x[[target_col]]
    X = data_x.drop(target_col, axis=1)

    # Scale the features and target
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    # Convert back to DataFrame (preserving column names and index)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    y_scaled_df = pd.DataFrame(y_scaled, columns=y.columns, index=y.index)

    # Concatenate scaled features and target
    data = pd.concat([X_scaled_df, y_scaled_df], axis=1)

    # Shift the target column to align with forecasting
    data[target_col+ '1'] = data_x[target_col]
    data[target_col] = data_x[target_col].shift(-forecast_horizon)

    columns_to_lag = [col for col in data.columns if col != target_col] # All columns except target

    # Create lagged features
    data_ls = create_lag_features(data, columns_to_lag, lags=lags)

    # Drop NaN values caused by shifting
    data_ls.dropna(inplace=True)

    if test_size_percent:
        test_size = int(len(data) * test_size)  # Convert percentage to absolute count

    train_df = data.iloc[:-test_size]
    test_df = data.iloc[-test_size:]

    X_train = train_df.drop(target_col, axis=1).values
    y_train = train_df[target_col].values
    X_test = test_df.drop(target_col, axis=1).values
    y_test = test_df[target_col].values

    X_train = X_train.reshape((X_train.shape[0], 1, 1, 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, 1, 1, X_test.shape[1]))

    return X_train, y_train, X_test, y_test, scaler_y
    # return data_ls, scaler_X, scaler_y


In [ ]:

# Perform robust testing with 50 random states
maes = []
rmses = []
cnn_all_predictions = []

save_dir = "/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXShuffle"

for i in range(100):
    file_path = os.path.join(save_dir, f"shuffled_data_{i+1}.csv")
    shuffled_data = pd.read_csv(file_path)

    X_train, y_train, X_test, y_test, scaler_y = prep_data_for_cnn(shuffled_data, 'CPIAUCSL', test_size=0.2, test_size_percent=True)
    model = build_model((1, 1, 1, X_train.shape[-1]))
    loss, predictions = evaluate_model(model, X_test, y_test)

    y_test_actual = inverse_transform(scaler_y, y_test)
    predictions_actual = inverse_transform(scaler_y, predictions)

    mae = mean_absolute_error(y_test_actual, predictions_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions_actual))

    # main(data, scaler_y)
    maes.append(mae)
    rmses.append(rmse)
    cnn_all_predictions.append(predictions)

# Calculate and print average MAE and RMSE
avg_mae = np.mean(maes)
avg_rmse = np.mean(rmses)

# Plotting the MAEs and RMSEs
plt.figure(figsize=(12, 6))

# Plot VAR MAEs
plt.plot(range(1, 101), maes, label='LSTM MAE', marker='o', linestyle='-', color='blue')

# Plot VAR RMSEs
plt.plot(range(1, 101), rmses, label='LSTM RMSE', marker='s', linestyle='-', color='green')

plt.title('LSTM: MAE and RMSE over 50 Random States')
plt.xlabel('Random State')
plt.ylabel('Error')
plt.legend()
plt.grid(True)
plt.show()

print(f'Average MAE: {avg_mae}')
print(f'Average RMSE: {avg_rmse}')


In [ ]:

# Fit SARIMA model

# Directory to save CSV files
save_dir = "/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXShuffle"

def train_and_evaluate_sarima_model(data, auto_arima_model_sarima,  test_size = 12, test_size_percent=False):

    if test_size_percent:
        test_size = int(len(data) * test_size)  # Convert percentage to absolute count

    train_df = data.iloc[:-test_size]
    test_df = data.iloc[-test_size:]
    # 1. ARIMA Model for CPIAUCSL

    # Extract CPIAUCSL series for ARIMA
    train_cpiauscl_arima = train_df['CPIAUCSL']
    test_cpiauscl_arima = test_df['CPIAUCSL']

    sarima_model = SARIMAX(train_cpiauscl_arima,
                           order=auto_arima_model_sarima.order,
                           seasonal_order=auto_arima_model_sarima.seasonal_order)

    sarima_model_fit = sarima_model.fit(disp=False) # disp=False to suppress convergence output

    # Forecast for test period (24 steps)
    sarima_forecast = sarima_model_fit.forecast(steps=test_size)

    # Evaluate SARIMA model
    mae_sarima = mean_absolute_error(test_cpiauscl_arima, sarima_forecast)
    rmse_sarima = np.sqrt(mean_squared_error(test_cpiauscl_arima, sarima_forecast))

    # model = VAR(train_df)
    # var_model = model.fit(lag_order)
    # forecast = var_model.forecast(train_df.values[-lag_order:], steps=12)
    # forecast_df = pd.DataFrame(forecast, columns=train_df.columns)
    # 
    # forecasted_cpiauscl = forecast_df['CPIAUCSL']
    # actual_cpiauscl = test_df['CPIAUCSL']

    mae = mean_absolute_error(actual_cpiauscl, forecasted_cpiauscl)
    rmse = np.sqrt(mean_squared_error(actual_cpiauscl, forecasted_cpiauscl))

    return mae_sarima, rmse_sarima, sarima_forecast

# Perform robust testing with 50 random states
var_maes = []
var_rmses = []
var_all_forecasts = []

for i in range(100):
    # Read shuffled data from CSV file
    file_path = os.path.join(save_dir, f"shuffled_data_{i+1}.csv")
    shuffled_data = pd.read_csv(file_path)

    # Create data DataFrame with selected columns
    # data = shuffled_data[['CPIAUCSL_1', 'UNRATE', 'RETAILx', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
    data = shuffled_data.copy()

    mae, rmse, forecasted_cpiauscl = train_and_evaluate_sarima_model(data, auto_arima_model_sarima, test_size = 0.2, test_size_percent=True)
    var_maes.append(mae)
    var_rmses.append(rmse)
    var_all_forecasts.append(forecasted_cpiauscl)



# ... (rest of your code for calculating averages and plotting)

In [ ]:
avg_sarima_mae = np.mean(var_maes)
avg_sarima_rmse = np.mean(var_rmses)

print(f'Average MAE: {avg_sarima_mae}')
print(f'Average RMSE: {avg_sarima_rmse}')


In [ ]:
# Compare performance at each step
conv_lstm_step_errors = np.array(cnn_all_predictions).reshape(100, 12)  # Reshape to (100, 12)
var_step_errors = np.array(var_all_forecasts).reshape(100, 12)

# Calculate mean absolute error for each step
conv_lstm_step_mae = np.mean(np.abs(conv_lstm_step_errors), axis=0)  # Average across 100 runs
var_step_mae = np.mean(np.abs(var_step_errors), axis=0)

# Compare MAE at each step
better_model_steps = ["ConvLSTM" if c_mae < v_mae else "SARIMA" for c_mae, v_mae in zip(conv_lstm_step_mae, var_step_mae)]

# Print the results
# Print the results with values
for i, (c_mae, v_mae) in enumerate(zip(conv_lstm_step_mae, var_step_mae)):
    better_model = "ConvLSTM" if c_mae < v_mae else "SARIMA"
    print(f"Step {i+1}: {better_model} performs better (ConvLSTM MAE: {c_mae:.4f}, SARIMA MAE: {v_mae:.4f})")

In [ ]:


# Calculate standard deviation for each step
conv_lstm_step_std = np.std(np.abs(conv_lstm_step_errors), axis=0)  # Standard deviation across 100 runs
var_step_std = np.std(np.abs(var_step_errors), axis=0)

# Plotting
plt.figure(figsize=(14, 7))

# Plot ConvLSTM MAE with shaded confidence interval
plt.plot(range(1, 13), conv_lstm_step_mae, label='ConvLSTM MAE', marker='o', linestyle='-', color='blue', alpha=0.8)
plt.fill_between(range(1, 13),
                 conv_lstm_step_mae - conv_lstm_step_std,
                 conv_lstm_step_mae + conv_lstm_step_std,
                 color='blue', alpha=0.2, label='ConvLSTM Confidence Interval')

# Plot VAR MAE with shaded confidence interval
plt.plot(range(1, 13), var_step_mae, label='VAR MAE', marker='x', linestyle='--', color='red', alpha=0.8)
plt.fill_between(range(1, 13),
                 var_step_mae - var_step_std,
                 var_step_mae + var_step_std,
                 color='red', alpha=0.2, label='SARIMA Confidence Interval')

# Customize the plot
plt.title('ConvLSTM vs SARIMA: Average MAE with Confidence Intervals over 12 Forecasting Steps', fontsize=16)
plt.xlabel('Forecasting Step', fontsize=14)
plt.ylabel('Average MAE', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



In [ ]:
# ... (your existing code)

# Calculate root mean squared error for each step
conv_lstm_step_rmse = np.sqrt(np.mean(conv_lstm_step_errors**2, axis=0))  # RMSE calculation
var_step_rmse = np.sqrt(np.mean(var_step_errors**2, axis=0))

# Compare RMSE at each step
better_model_steps = ["ConvLSTM" if c_rmse < v_rmse else "SARIMA" for c_rmse, v_rmse in zip(conv_lstm_step_rmse, var_step_rmse)]

# Print the results with values
for i, (c_rmse, v_rmse) in enumerate(zip(conv_lstm_step_rmse, var_step_rmse)):
    better_model = "ConvLSTM" if c_rmse < v_rmse else "SARIMA"
    print(f"Step {i+1}: {better_model} performs better (ConvLSTM RMSE: {c_rmse:.4f}, SARIMA: {v_rmse:.4f})")

In [ ]:
plt.figure(figsize=(12, 6))
# conv_lstm_step_rmse = np.sqrt(np.mean(conv_lstm_step_errors**2, axis=0))  # RMSE calculation
# var_step_rmse = np.sqrt(np.mean(var_step_errors**2, axis=0))

# Plot ConvLSTM and VAR `RMSEs'
plt.plot(range(1, 13), conv_lstm_step_rmse, label='ConvLSTM RMSE', marker='o', linestyle='-', color='blue', alpha=0.8)
plt.plot(range(1, 13), var_step_rmse, label='SARIMA RMSE', marker='x', linestyle='--', color='red', alpha=0.8)

plt.title('ConvLSTM vs SARIMA: Average RMSE over 12 Forecasting Steps')
plt.xlabel('Forecasting Step')
plt.ylabel('Average RMSE')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

